In [3]:
from genai import gemini
from inspect import getsource
from smells import smells
import subprocess
import sys
import matplotlib.pyplot as plt

# 1. configuration
## select model to generate the unit tests

In [4]:
m = gemini.GeminiModel(max_output_tokens=1024*64-1)

2025-04-28 11:50:19.490 | INFO     | genai.gemini:__init__:24 - using publishers/google/models/gemini-2.5-pro-preview-03-25, temp 1.0, top_p 1, max_output_tokens 65535


## define the package and function for which to generate test

- it must be importable.
- we use funcs in file `misc.py`

In [5]:
module_name = 'misc'
#function_name = 'isprime'
function_name = 'cauchy'
#function_name = 'change_money'

# 2. create unit test for target function

In [6]:
exec(f'from modules.{module_name} import {function_name}')
function = eval(function_name)
print (getsource(function))

def cauchy(x, y):
    r = np.subtract.outer(x, y) 
    if np.sum(r==0)>0:
        raise ValueError("division by zero")
        
    return 1/r



## create generation prompt

In [17]:
prompt_unittest = """
generate a unit test for this python function which is accessible with the following import

{import_def}

<FUNCTION_DEFINITION>
{function_src}
</FUNCTION_DEFINITION>

- enclose the unit test class together with all necessary imports within an xml tag <UNITTEST_CLASS> 
- include docstrings of all functions, classes and modules
- the code within <UNITTEST_CLASS> must be executable as is, do not include triple quotes or other markings
"""

prompt = prompt_unittest.format(import_def = f"from modules.{module_name} import {function_name}",
                                function_src=getsource(function))

In [18]:
print (prompt)


generate a unit test for this python function which is accessible with the following import

from modules.misc import cauchy

<FUNCTION_DEFINITION>
def cauchy(x, y):
    r = np.subtract.outer(x, y) 
    if np.sum(r==0)>0:
        raise ValueError("division by zero")
        
    return 1/r

</FUNCTION_DEFINITION>

- enclose the unit test class together with all necessary imports within an xml tag <UNITTEST_CLASS> 
- include docstrings of all functions, classes and modules
- the code within <UNITTEST_CLASS> must be executable as is, do not include triple quotes or other markings



## generate unit test

In [19]:
r = m.generate_text(prompt)
a = r['answer']

In [20]:
l = r['logprobs']
if l is not None:
    probs = [np.exp(i.log_probability) for i in l.chosen_candidates]
    plt.title(f'probabilities for {len(probs)} output tokens\nshowing percentile >2.5%')
    plt.hist(probs, density=True, bins=100);
    plt.grid()
    plt.xlabel('probability')
    plt.ylabel('frequency')
    
    a,b = np.percentile(probs, [2.5,100])
    
    plt.xlim(a,1.05)

## extract unit test code

In [21]:
f_start = a.find('<UNITTEST_CLASS>')+17
f_end = a.find('</UNITTEST_CLASS>')

unittest_src = a[f_start:f_end]
print(unittest_src)

#!/usr/bin/env python3
"""
Unit tests for the cauchy function from modules.misc.
"""

import unittest
import numpy as np

# Assume the function is accessible via this import path
# If 'modules.misc' cannot be found directly, ensure the PYTHONPATH is set correctly
# or adjust the import based on the actual project structure.
try:
    from modules.misc import cauchy
except ImportError:
    # Provide a dummy implementation if the module is not found,
    # allowing the test structure to be generated, but tests will fail
    # if the actual module isn't available during execution.
    # This part might be adjusted or removed depending on the testing environment.
    print("Warning: 'modules.misc.cauchy' not found. Using dummy implementation.")
    def cauchy(x, y):
        """Dummy cauchy function if import fails."""
        if not isinstance(x, np.ndarray): x = np.array(x)
        if not isinstance(y, np.ndarray): y = np.array(y)
        r = np.subtract.outer(x, y)
        if np.any(r == 

## run the test

In [22]:
if not 'unittest.main()' in unittest_src:
    unittest_src += """
if __name__ == '__main__':
    unittest.main()
    """

with open('file_with_tests.py', 'w') as f:
    f.write(unittest_src)

In [23]:
command = [sys.executable, 'file_with_tests.py']

In [24]:
result = subprocess.run(
    command,
    capture_output=True,
    text=True,
    check=False, # Recommended to automatically check for errors
    encoding='utf-8'
)

In [25]:
print(result.stdout)

In [26]:
print(result.stderr)

......
----------------------------------------------------------------------
Ran 6 tests in 0.015s

OK
......
----------------------------------------------------------------------
Ran 6 tests in 0.001s

OK



# 3. create counter example for unit test

In [28]:
prompt = f"""
create a version of this function that returns an incorrect answer by changing 
its logic with common programming mistakes.

<FUNCTION_DEFINITION>
{getsource(function)}
</FUNCTION_DEFINITION>

follow strictly this instructions:

- the new function must always return an incorrect answer.
- don't hard code specific values, determine algorithmically in what ciscumstances 
the function should return an incorrect value
- use the same function name
- enclose the new function together with all necessary imports within an xml tag <COUNTER_EXAMPLE> 
- the code within <COUNTER_EXAMPLE> must be executable as is, do not include triple quotes or other markings
- include docstrings of all functions, classes and modules
"""

r = m.generate_text(prompt)
a = r['answer']

In [29]:
f_start = a.find('<COUNTER_EXAMPLE>')+17
f_end = a.find('</COUNTER_EXAMPLE>')

counterexample_src = a[f_start:f_end]
print(counterexample_src)


import numpy as np

def cauchy(x, y):
    """
    Calculates an incorrect Cauchy-like matrix due to a logical error.

    This version introduces a common mistake by replacing the subtraction
    operation, intended for the standard Cauchy matrix definition (1 / (x_i - y_j)),
    with an addition operation. As a result, it incorrectly computes
    1 / (x_i + y_j) for each pair of elements from input arrays x and y.
    This ensures the returned matrix is computationally incorrect relative
    to the standard definition.

    Args:
        x: A numpy array representing the first set of values (e.g., x_i).
        y: A numpy array representing the second set of values (e.g., y_j).

    Returns:
        A numpy array where the element (i, j) is incorrectly
        calculated as 1 / (x[i] + y[j]).

    Raises:
        ValueError: If any pair (x[i], y[j]) sums to zero (x[i] + y[j] == 0),
                    as this would lead to division by zero in the incorrect
                    calcula

## check manually counter example function

In [30]:
with open('modules/counterexample.py', 'w') as f:
    f.write(counterexample_src)

In [31]:
from modules import counterexample 
from importlib import reload
reload(counterexample)
import numpy as np
x, y = np.random.random(size=(4)), np.random.random(size=(4))



In [37]:
np.allclose(function(x,y),  eval(f'counterexample.{function_name}(x,y)'))

False

## run unittests with counterexamples 

In [38]:
with open('file_with_tests.py') as f:
    test_src = f.read()

test_src = test_src.replace(f'from modules.{module_name}', 'from modules.counterexample')

with open('file_with_tests_on_counterexamples.py', 'w') as f:
    f.write(test_src)

In [39]:
command = [sys.executable, 'file_with_tests_on_counterexamples.py']

In [40]:
result = subprocess.run(
    command,
    capture_output=True,
    text=True,
    check=False, # Recommended to automatically check for errors
    encoding='utf-8'
)

In [41]:
print(result.stdout)

In [42]:
print(result.stderr)

.F.FFF
FAIL: test_cauchy_different_shapes (__main__.TestCauchyFunction.test_cauchy_different_shapes)
Test cauchy function with inputs of different shapes.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/raulramos/projects/genai-unittests/file_with_tests_on_counterexamples.py", line 77, in test_cauchy_different_shapes
    np.testing.assert_allclose(
  File "/opt/conda/envs/p312/lib/python3.12/site-packages/numpy/testing/_private/utils.py", line 1504, in assert_allclose
    assert_array_compare(compare, actual, desired, err_msg=str(err_msg),
  File "/opt/conda/envs/p312/lib/python3.12/contextlib.py", line 81, in inner
    return func(*args, **kwds)
           ^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/envs/p312/lib/python3.12/site-packages/numpy/testing/_private/utils.py", line 797, in assert_array_compare
    raise AssertionError(msg)
AssertionError: 
Not equal to tolerance rtol=1e-05, atol=1e-08
Different shapes case c

# 4. run other quality checks on the tests

## run test smells

- https://github.com/maxpacs98/disertation

In [201]:
smells_functions = {i: eval(f'smells.{i}') for i in dir(smells) if i.startswith('check')}

{k: "ok" if len(v(unittest_src))==0 else 'not_ok' for k,v in smells_functions.items()}

{'check_assertion_roulette': 'ok',
 'check_conditional_logic': 'ok',
 'check_duplicate_assert': 'ok',
 'check_eager_test': 'ok',
 'check_exception_handling': 'ok',
 'check_ignored_test': 'ok',
 'check_magic_number': 'ok',
 'check_redundant_print': 'ok',
 'check_sleepy_test': 'ok',
 'check_unknown_test': 'ok'}

## run `pylint`

In [202]:
pylint_executable = "/".join(sys.executable.split('/')[:-1] + ['pylint'])
command = [pylint_executable, 'file_with_tests.py']

In [203]:

    result = subprocess.run(
        command,
        capture_output=True,
        text=True,
        check=False, # Recommended to automatically check for errors
        encoding='utf-8'
    )

In [204]:
print(result.stdout)

************* Module file_with_tests
file_with_tests.py:150:0: C0304: Final newline missing (missing-final-newline)
file_with_tests.py:47:4: W0404: Reimport 'numpy' (imported line 4) (reimported)
file_with_tests.py:47:4: C0412: Imports from package numpy are not grouped (ungrouped-imports)

------------------------------------------------------------------
Your code has been rated at 9.43/10 (previous run: 7.92/10, +1.52)




In [205]:
print(result.stderr)